## Spectral sequence of a ring

In [7]:
import plot
path_ring = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\mix\S0_AdamsSS_t261.db'
data_ring = plot.load_ss_ring(path_ring)
plot.export_ss_ring(data_ring)

## Spectral sequence of a module

In [ ]:
import plot
path_ring = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\tmf_AdamsSS.db'
path_mod = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\tmf_C2_AdamsSS.db'
data_ring, data_mod = plot.load_ss_mod(path_ring, path_mod)
plot.export_ss_mod(data_ring, data_mod)

## Homotopy of a ring

In [1]:
import plot
path_ring = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\main\S0_AdamsSS_t261.db'
data = plot.load_pi_ring(path_ring)
plot.export_pi_ring(data)

## Homotopy of a module

In [70]:
import plot
path_ring = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\debug\S0_AdamsSS_t100.db'
path_mod = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\debug\C2_AdamsSS_t100.db'
data_ring, data_mod = plot.load_pi_mod(path_ring,path_mod)
plot.export_pi_mod(data_ring, data_mod)

## Homotopy exact sequence

In [3]:
import plot
path_ring = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\debug\S0_AdamsSS_t100.db'
path_mod = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\debug\Csigma_AdamsSS_t100.db'
data_ring, data_mod = plot.load_pi_mod(path_ring,path_mod)
plot.export_pi_exact(data_ring, data_mod)

## Spectral sequence of a module with prod_hi

In [2]:
import plot
path = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\RP1_383_Adams_res.db'
data = plot.load_ss_from_res(path)
plot.export_ss_from_res(data)

In [3]:
data

{'basis': [[0, 1],
  [524288, 1],
  [524289, 1],
  [524290, 1],
  [524291, 1],
  [524292, 1],
  [524293, 1],
  [524294, 1],
  [1048576, 1],
  [1048577, 1],
  [1048578, 1],
  [1048579, 1],
  [1048580, 1],
  [1048581, 1],
  [1048582, 1],
  [1048583, 1],
  [1048584, 1],
  [1048585, 1],
  [1048586, 1],
  [1048587, 1],
  [1048588, 1],
  [1048589, 1],
  [1048590, 1],
  [1048591, 1],
  [1048592, 1],
  [1048593, 1],
  [1048594, 1],
  [1048595, 1],
  [1048596, 1],
  [1572864, 1],
  [1572865, 1],
  [1572866, 1],
  [1572867, 1],
  [1572868, 1],
  [1572869, 1],
  [1572870, 1],
  [1572871, 1],
  [1572872, 1],
  [1572873, 1],
  [1572874, 1],
  [1572875, 1],
  [1572876, 1],
  [1572877, 1],
  [1572878, 1],
  [1572879, 1],
  [1572880, 1],
  [1572881, 1],
  [1572882, 1],
  [1572883, 1],
  [1572884, 1],
  [1572885, 1],
  [1572886, 1],
  [1572887, 1],
  [1572888, 1],
  [1572889, 1],
  [1572890, 1],
  [1572891, 1],
  [1572892, 1],
  [1572893, 1],
  [1572894, 1],
  [1572895, 1],
  [1572896, 1],
  [1572897, 

## Update website pages

In [29]:
import plot
complexes = ["S0", "C2", "Ceta", "Cnu", "Csigma"]

dir_db = 'C:/Users/lwnpk/Documents/Projects/algtop_cpp_build/bin/Release/mix100'
dir_website = "C:/Users/lwnpk/OneDrive/Projects/HTML/WayneLin92.github.io/ss-fb42729d/mix100"

for complex in complexes:
    path_ring = f'{dir_db}/S0_AdamsSS_t100.db'
    path_mod =  f'{dir_db}/{complex}_AdamsSS_t100.db'
    if complex == "S0":
        path_html = f"{dir_website}/S0_AdamsE2_t100/index.html"
        path_js = f"{dir_website}/S0_AdamsE2_t100/data.js"
        data_ring = plot.load_ss_ring(path_ring)
        plot.export_ss_ring(data_ring, path_html, path_js)

        path_html = f"{dir_website}/S0_pi_t100/index.html"
        path_js = f"{dir_website}/S0_pi_t100/data.js"
        data = plot.load_pi_ring(path_ring)
        plot.export_pi_ring(data, path_html, path_js)
    else:
        path_html = f"{dir_website}/{complex}_AdamsE2_t100/index.html"
        path_js = f"{dir_website}/{complex}_AdamsE2_t100/data.js"
        data_ring, data_mod = plot.load_ss_mod(path_ring, path_mod)
        plot.export_ss_mod(data_ring, data_mod, path_html, path_js)

        path_html = f"{dir_website}/{complex}_pi_t100/index.html"
        path_js = f"{dir_website}/{complex}_pi_t100/data.js"
        data_ring, data_mod = plot.load_pi_mod(path_ring,path_mod)
        plot.export_pi_mod(data_ring, data_mod, path_html, path_js)

        path_html = f"{dir_website}/{complex}_exact_t100/index.html"
        path_js = f"{dir_website}/{complex}_exact_t100/data.js"
        data_ring, data_mod = plot.load_pi_mod(path_ring,path_mod)
        plot.export_pi_exact(data_ring, data_mod, path_html, path_js)